In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 4.5 MB/s 
     |████████████████████████████████| 408 kB 58.5 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.5 which is incompatible.
tensorflow-metadata 1.9.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.5 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.5 which is incompatible.
google-api-core 1.31.6 requires protobuf<4.0.0dev,>=3.12.0; python_version > "3", but you have protob

# Create Torch Dataset

In [ ]:
import math
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from IPython.display import clear_output
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

class TSPDataset(Dataset):
    
    def __init__(self, num_nodes, num_samples, random_seed=111):
        super(TSPDataset, self).__init__()
        torch.manual_seed(random_seed)

        self.data_set = []
        for l in tqdm(range(num_samples)):
            x = torch.FloatTensor(2, num_nodes).uniform_(0, 1)
            self.data_set.append(x)

        self.size = len(self.data_set)

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.data_set[idx]


In [ ]:
# train_size = 10000
# val_size = 1000

# train_9_dataset = TSPDataset(9, train_size)
# val_9_dataset   = TSPDataset(9, val_size)

# torch.save(train_9_dataset, 'train_9_dataset.pt')
# torch.save(val_9_dataset, 'val_9_dataset.pt')


# train_21_dataset = TSPDataset(21, train_size)
# val_21_dataset   = TSPDataset(21, val_size)

# torch.save(train_21_dataset, 'train_21_dataset.pt')
# torch.save(val_21_dataset, 'val_21_dataset.pt')

100%|██████████| 1000/1000 [00:00<00:00, 139791.49it/s]


## Load Dataset Object

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/MyDrive/VRP_research')
os.getcwd()


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


'/content/drive/.shortcut-targets-by-id/1R2Lj1xvAELBwdRrGDZQ4E3hOev4DnDBa/VRP research'

In [ ]:
train_9_dataset = torch.load('train_9_dataset.pt')
val_9_dataset = torch.load('val_9_dataset.pt')

train_21_dataset = torch.load('train_21_dataset.pt')
val_21_dataset   =  torch.load('val_21_dataset.pt')

In [ ]:
len(train_9_dataset.data_set[0].T)

21

# Google OR for VRP with pickup and deliveries

In [ ]:
"""Simple Pickup Delivery Problem (PDP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(locations):
    """Stores the data for the problem.
    args:
    locations: one sample's transpose from the TSP dataset list, train_9_dataset.data_set[0].T
               e.g. n = 3, tensor: [[1,0.4,0.5], [0.1,0.5,0.7]], 3 coordinate pairs
    """

    n = len(locations)
    data = {}    
    data['locations'] = locations
    data['distance_matrix'] = compute_euclidean_distance_matrix(data['locations'])

    pickup_idx = [i for i in range(1, int((n+1) / 2))]
    dropoff_idx = [j for j in range(int((n+1)/2), n)]
    data['pickups_deliveries'] = [list(pair) for pair in zip(pickup_idx,dropoff_idx)]
    # data['pickups_deliveries'] = [
    #     [1, 5],
    #     [2, 6],
    #     [3, 7],
    #     [4, 8]
    # ]
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    # print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_idx = [] 
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            route_idx.append(manager.IndexToNode(index))
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        # print(plan_output)
        total_distance += route_distance
    
    # scale back the total distance (we scale by multiplying 1000 before for the OR int input requirement)
    total_distance = total_distance / 1000

    print('Total Distance of all routes: {}m'.format(total_distance))
    return total_distance, route_idx


def main_solver(locations):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(locations)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        30000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_distance, route_idx = print_solution(data, manager, routing, solution)
        return total_distance, route_idx

# if __name__ == '__main__':
#     main()


In [ ]:
main_solver(val_9_dataset.data_set[0].T)

Total Distance of all routes: 2.578m


(2.578, [0, 3, 2, 7, 6, 1, 4, 8, 5])

### Save solution dictionary as pkl: {id: {'distance':xxxx, 'idx':[xxxxx] } }

In [ ]:
sol = {}
sol_distances = []
sol_idxs = []

for sol_id, data_set in enumerate(val_9_dataset.data_set):
  sol[sol_id] = {}
  distance, route_idx = main_solver(data_set.T)
  sol[sol_id]['idx'] = route_idx
  sol[sol_id]['distance'] = distance

  sol_distances.append(distance)
  sol_idxs.append(route_idx)


In [ ]:
print('Mean of valuation set tour length:{}'.format(np.array(sol_distances).mean()))
print('Sample solution route sequence: {}'.format(sol_idxs[0]))

Mean of valuation set tour length:2.628837
Sample solution route sequence: [0, 3, 2, 7, 6, 1, 4, 8, 5]


In [ ]:
import pickle

# with open('GoogleOR_sol_val_9.pkl', 'wb') as f:
#     pickle.dump(sol, f)
        
with open('GoogleOR_sol_val_9.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

### load saved solution as dict

In [ ]:
dist = []

for sol_id, sol in loaded_dict.items():
  dist.append(sol['distance'])

print('Mean of valuation set tour length:{}'.format(np.array(dist).mean()))
print('Sample solution route sequence: {}'.format(loaded_dict[0]['idx']))

Mean of valuation set tour length:2.628837
Sample solution route sequence: [0, 3, 2, 7, 6, 1, 4, 8, 5]
